In [78]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.svm import SVC

In [79]:
data_path = "data/"

seeds = pd.read_csv(data_path + 'MNCAATourneySeeds.csv')
teams = pd.read_csv(data_path + 'MTeams.csv')
reg_szn = pd.read_csv(data_path + 'MRegularSeasonDetailedResults.csv')
tourney_seeds = pd.read_csv(data_path + '2024_tourney_seeds.csv')
tourney_results = pd.read_csv(data_path + 'MNCAATourneyDetailedResults.csv')

In [80]:
#Shuffle classes
results = tourney_results[['Season', 'WTeamID', 'LTeamID']].rename(columns={'WTeamID': 'Team1', 'LTeamID': 'Team2'})
shuffled_games = results.sample(frac=1)
win_2 = shuffled_games[:results.shape[0]//2].rename(columns={'Team1': 'Team2', 'Team2': 'Team1'})[['Season', 'Team1', 'Team2']]
win_2['won/lost'] = 0
win_1 = shuffled_games.sample(frac=1)[results.shape[0]//2:]
win_1['won/lost'] = 1
train_data = pd.concat((win_2, win_1))

In [81]:
def convert_seed(s):
    """
    converts the the string s given by the seed of a particular
    team in the seeds DataFrame into the integer equivalent so
    that the seeding of a team can be accounted for in the model.
    """
    if s[1] == '0':
        return int(s[2])
    else:
        return int(s[1:3])

In [82]:
#Unused features
Drop_cols = ['DayNum', 'LTeamID', 'NumOT', 'WStl', 'WBlk', 'WPF', 'LAst', 'LStl', 'LBlk', 'LPF', 'LFTA', 'LFTM']

In [83]:
#aggregate team stats by wins
w_updated = reg_szn.rename(columns={'WTeamID': 'TeamID'})
w_stat_totals = w_updated.groupby(['Season', 'TeamID']).sum()
w_stat_totals = w_stat_totals.drop(columns=Drop_cols)
w_stat_totals = w_stat_totals.rename(columns={'WScore': 'Pts', 'LScore': 'OPts',
                                    'WFGM': 'FGM', 'WFGA': 'FGA', 'WFGM3': 'FGM3', 
                                    'WFGA3': 'FGA3', 'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast',
                                    'WTO': 'TO', 'LFGM': 'OFGM', 'LFGA': 'OFGA',
                                    'LFGM3': 'OFGM3', 'LFGA3': 'OFGA3', 'WFTM': 'FTM',
                                    'WFTA': 'FTA', 'LOR': 'OOR', 'LDR': 'ODR', 'LTO': 'OTO'})

/var/folders/l1/4__m_kt95kl59cqwpjcgh6d80000gn/T/ipykernel_9117/3316018407.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  w_stat_totals = w_updated.groupby(['Season', 'TeamID']).sum()


In [84]:
updated = reg_szn.rename(columns={'LTeamID': 'TeamID'})
l_stat_totals = updated.groupby(['Season', 'TeamID']).sum()
Drop_co = ['DayNum', 'WTeamID', 'NumOT', 'LStl', 'LBlk', 'LPF', 'WAst', 'WStl', 'WBlk', 'WPF', 'WFTA', 'WFTM']
l_stat_totals = l_stat_totals.drop(columns=Drop_co)
l_stat_totals = l_stat_totals.rename(columns={'WScore': 'OPts', 'LScore': 'Pts',
                                    'WFGM': 'OFGM', 'WFGA': 'OFGA', 'WFGM3': 'OFGM3', 
                                    'WFGA3': 'OFGA3', 'WOR': 'OOR', 'WDR': 'ODR', 'LAst': 'Ast',
                                    'WTO': 'OTO', 'LFGM': 'FGM', 'LFGA': 'FGA',
                                    'LFGM3': 'FGM3', 'LFGA3': 'FGA3', 'LFTM': 'FTM',
                                    'LFTA': 'FTA', 'LOR': 'OR', 'LDR': 'DR', 'LTO': 'TO'})
index1 = pd.MultiIndex.from_tuples([(2015, 1246)], names=["Season", "TeamID"])
index2 = pd.MultiIndex.from_tuples([(2014, 1455)], names=["Season", "TeamID"])
index3 = pd.MultiIndex.from_tuples([(2021, 1211)], names=["Season", "TeamID"])
kentucky_tot = pd.DataFrame([[0]*l_stat_totals.shape[1]], columns = list(l_stat_totals.columns), index = index1)
wichita_tot = pd.DataFrame([[0]*l_stat_totals.shape[1]], columns = list(l_stat_totals.columns), index = index2)
gonzaga_tot = pd.DataFrame([[0]*l_stat_totals.shape[1]], columns = list(l_stat_totals.columns), index = index3)
l_stat_totals = pd.concat((l_stat_totals, kentucky_tot))
l_stat_totals = pd.concat((l_stat_totals, wichita_tot))
l_stat_totals = pd.concat((l_stat_totals, gonzaga_tot))

/var/folders/l1/4__m_kt95kl59cqwpjcgh6d80000gn/T/ipykernel_9117/3276248575.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  l_stat_totals = updated.groupby(['Season', 'TeamID']).sum()


In [85]:
stat_totals = l_stat_totals.add(w_stat_totals)

In [86]:
#aggregating number of wins and losses by team
w_tot_games = w_updated.groupby(['Season', 'TeamID']).count()

l_tot_games = updated.groupby(['Season', 'TeamID']).count()
idx1 = pd.MultiIndex.from_tuples([(2015, 1246)], names=["Season", "TeamID"])
idx2 = pd.MultiIndex.from_tuples([(2014, 1455)], names=["Season", "TeamID"])
idx3 = pd.MultiIndex.from_tuples([(2021, 1211)], names=["Season", "TeamID"])
kentucky_num = pd.DataFrame([[0]*l_tot_games.shape[1]], columns = list(l_tot_games.columns), index = idx1)
#Kentucky was undefeated entering the tournament in 2015
wichita_num = pd.DataFrame([[0]*l_tot_games.shape[1]], columns = list(l_tot_games.columns), index = idx2)
#Wichita State was undefeated entering the tournament in 2014
gonzaga_num = pd.DataFrame([[0]*l_tot_games.shape[1]], columns = list(l_tot_games.columns), index = idx3)
#Gonzaga was undefeated entering the tournament in 2021
Drops = ['DayNum', 'NumOT', 'WStl', 'WBlk', 'WPF', 'LAst', 'LStl', 'LBlk', 'LPF', 'LFTA', 'LFTM', 'TeamID', 'WTeamID', 'WLoc']
l_tot_games = pd.concat((l_tot_games, kentucky_num))
l_tot_games = pd.concat((l_tot_games, wichita_num))
l_tot_games = pd.concat((l_tot_games, gonzaga_num))

In [87]:
l_tot_games['losses'] = l_tot_games['DayNum']
l_tot_games['wins'] = 0
w_tot_games['wins'] = w_tot_games['DayNum']
w_tot_games['losses'] = 0

In [88]:
Drops = ['DayNum', 'NumOT', 'WStl', 'WBlk', 'WPF', 'LAst', 'LStl', 'LBlk', 'LPF', 'LFTA', 'LFTM', 'WTeamID', 'WLoc', 'LTeamID']
games_count = l_tot_games + w_tot_games
games_count = games_count.rename(columns={'WScore': 'Pts', 'LScore': 'OPts',
                                    'WFGM': 'FGM', 'WFGA': 'FGA', 'WFGM3': 'FGM3', 
                                    'WFGA3': 'FGA3', 'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast',
                                    'WTO': 'TO', 'LFGM': 'OFGM', 'LFGA': 'OFGA',
                                    'LFGM3': 'OFGM3', 'LFGA3': 'OFGA3', 'WFTM': 'FTM',
                                    'WFTA': 'FTA', 'LOR': 'OOR', 'LDR': 'ODR', 'LTO': 'OTO'})
games_count = games_count.drop(columns=Drops).dropna()
#drops the N/A values that represents a team that finished a season with 0 wins, as this team is assumed to have not made the tournament.

In [89]:
team_averages = stat_totals/games_count[games_count.columns[:-2]]

In [90]:
team_averages = team_averages.merge(games_count[['wins', 'losses']], left_index=True, right_index=True)

In [91]:
seeds['seed_conv'] = seeds['Seed'].apply(convert_seed)

In [92]:
seeds = seeds[['Season', 'TeamID', 'seed_conv']]

In [93]:
train_data = train_data.merge(seeds, left_on=['Season', 'Team1'], right_on=['Season', 'TeamID']).drop(columns=['TeamID']).rename(columns={'seed_conv': 'Seed_1'})

In [94]:
train_data = train_data.merge(seeds, left_on=['Season', 'Team2'], right_on=['Season', 'TeamID']).drop(columns=['TeamID']).rename(columns={'seed_conv': 'Seed_2'})

In [95]:
def get_team(team_id):
    return teams[teams['TeamID']==team_id]

In [96]:
train_data

,Season,Team1,Team2,won/lost,Seed_1,Seed_2
0,2013,1393,1276,0,4,4
1,2013,1433,1276,0,5,4
2,2013,1393,1231,1,4,1
3,2013,1396,1231,0,9,1
4,2009,1393,1328,0,3,2
...,...,...,...,...,...,...
1310,2014,1273,1181,1,14,3
1311,2013,1195,1361,1,15,7
1312,2006,1211,1462,1,3,14
1313,2007,1139,1268,1,5,4


In [20]:
team_averages['FT'] = team_averages['FTM']/team_averages['FTA']*100
team_averages['FG'] = team_averages['FGM']/team_averages['FGA']*100

In [21]:
team_averages = team_averages[['Pts', 'OPts', 'FG', 'FGM3', 'FT', 'Ast', 'DR', 'OR', 'TO', 'OTO', 'wins', 'losses']]
team_averages = team_averages.reset_index()

In [22]:
train_data = train_data.merge(team_averages, left_on=['Season', 'Team1'], right_on=['Season', 'TeamID']).drop(columns=['TeamID'])
train_data.columns = np.append(train_data.columns[:6], train_data.columns[6:]+'_1')
train_data = train_data.merge(team_averages, left_on=['Season', 'Team2'], right_on=['Season', 'TeamID']).drop(columns=['TeamID'])
train_data.columns = np.append(train_data.columns[:18], train_data.columns[18:]+'_2')
train_data['W/L'] = train_data['won/lost']
train_data = train_data.drop(columns=['won/lost'])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(train_data[train_data.columns[:29]], train_data['W/L'])

In [24]:
X_train

,Season,Team1,Team2,Seed_1,Seed_2,Pts_1,OPts_1,FG_1,FGM3_1,FT_1,...,FG_2,FGM3_2,FT_2,Ast_2,DR_2,OR_2,TO_2,OTO_2,wins_2,losses_2
296,2014,1257,1246,4,8,82.117647,60.970588,47.373494,8.058824,65.871122,...,45.123880,5.088235,68.235294,11.500000,26.500000,14.647059,12.205882,11.000000,24.0,10.0
851,2010,1325,1397,14,6,73.441176,68.558824,42.586912,7.235294,73.308271,...,45.258842,5.878788,66.856330,14.333333,25.000000,11.727273,13.242424,16.757576,25.0,8.0
453,2023,1231,1274,4,5,75.030303,68.454545,48.957247,5.666667,71.230503,...,48.584906,7.593750,77.622378,14.937500,22.906250,8.968750,10.937500,12.187500,25.0,7.0
168,2003,1400,1393,1,3,78.857143,68.678571,44.851259,5.857143,71.471471,...,47.006652,5.241379,69.343066,14.965517,26.896552,14.310345,13.620690,14.448276,24.0,5.0
237,2015,1374,1417,6,11,69.406250,59.718750,48.256881,4.218750,70.457698,...,44.058121,6.212121,67.571429,13.909091,25.666667,12.303030,11.878788,12.606061,20.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,2018,1138,1112,13,4,84.696970,76.515152,47.380730,9.636364,69.598765,...,50.541516,6.911765,76.027397,15.235294,26.617647,10.029412,12.294118,11.647059,27.0,7.0
869,2010,1143,1181,8,1,77.969697,68.393939,47.744946,6.575758,75.699558,...,44.015633,7.558824,76.288660,13.941176,24.441176,14.617647,11.117647,14.794118,29.0,5.0
1219,2006,1228,1102,4,13,69.967742,58.096774,45.632184,7.354839,63.375224,...,48.119122,8.607143,74.637681,14.178571,17.785714,6.785714,10.928571,14.964286,22.0,6.0
77,2019,1199,1436,4,13,74.911765,67.117647,44.304435,7.500000,74.063800,...,45.267959,7.812500,75.314465,11.500000,25.718750,8.937500,11.000000,12.875000,26.0,6.0


In [25]:
clf = LogisticRegression(random_state=0)
clf.fit(X_train[X_train.columns[3:]], y_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [26]:
#clf = RandomForestClassifier(max_depth=5, random_state=0)
#clf.fit(X_train[X_train.columns[3:]], y_train)

In [27]:
#clf = SVC(gamma='auto')
#clf.fit(X_train[X_train.columns[3:]], y_train)

In [28]:
predictions = clf.predict(X_test[X_test.columns[3:]])

In [29]:
(y_test==predictions).mean()

0.6443768996960486

In [30]:
(y_test==predictions).mean()

0.6443768996960486

In [31]:
tourney_2024 = team_averages[team_averages['Season']==2024]

In [32]:
tourney_teams = tourney_2024.merge(seeds, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID']).rename(columns={'seed_conv': 'Seed'})

In [33]:
team_1 = tourney_teams[tourney_teams['TeamID']==1103]

In [34]:
team_2 = tourney_teams[tourney_teams['TeamID']==1104]

In [35]:
game = team_1.merge(team_2, on='Season').drop(columns=['Season', 'TeamID_x', 'TeamID_y'])

In [36]:
cols = np.array([])
for c in game.columns:
    if c[-1] == 'x':
        cols = np.append(cols, c[:-1] + '1')
    else:
        cols = np.append(cols, c[:-1] + '2') 

In [37]:
game.columns = cols

In [38]:
columns_1 = np.append(game.columns[12], game.columns[25])
columns_2 = np.append(columns_1, game.columns[:12])
columns_3 = np.append(columns_2, game.columns[13:25])

In [39]:
columns_3

array(['Seed_1', 'Seed_2', 'Pts_1', 'OPts_1', 'FG_1', 'FGM3_1', 'FT_1',
       'Ast_1', 'DR_1', 'OR_1', 'TO_1', 'OTO_1', 'wins_1', 'losses_1',
       'Pts_2', 'OPts_2', 'FG_2', 'FGM3_2', 'FT_2', 'Ast_2', 'DR_2',
       'OR_2', 'TO_2', 'OTO_2', 'wins_2', 'losses_2'], dtype=object)

In [40]:
list(columns_3)

['Seed_1',
 'Seed_2',
 'Pts_1',
 'OPts_1',
 'FG_1',
 'FGM3_1',
 'FT_1',
 'Ast_1',
 'DR_1',
 'OR_1',
 'TO_1',
 'OTO_1',
 'wins_1',
 'losses_1',
 'Pts_2',
 'OPts_2',
 'FG_2',
 'FGM3_2',
 'FT_2',
 'Ast_2',
 'DR_2',
 'OR_2',
 'TO_2',
 'OTO_2',
 'wins_2',
 'losses_2']

In [41]:
game = game[list(columns_3)]

In [42]:
clf.predict(game)

array([0])

In [43]:
clf.predict_proba(game)[0]

array([0.888217, 0.111783])

In [44]:
def predict_outcome(team_1, team_2):
    first = tourney_teams[tourney_teams['TeamID']==team_1]
    second = tourney_teams[tourney_teams['TeamID']==team_2]
    game = first.merge(second, on='Season').drop(columns=['Season', 'TeamID_x', 'TeamID_y'])
    cols = np.array([])
    for c in game.columns:
        if c[-1] == 'x':
            cols = np.append(cols, c[:-1] + '1')
        else:
            cols = np.append(cols, c[:-1] + '2')
    game.columns = cols
    columns_1 = np.append(game.columns[12], game.columns[25])
    columns_2 = np.append(columns_1, game.columns[:12])
    columns_3 = np.append(columns_2, game.columns[13:25])
    game = game[list(columns_3)]
    return clf.predict(game)[0]

In [45]:
tourney_teams

,Season,TeamID,Pts,OPts,FG,FGM3,FT,Ast,DR,OR,TO,OTO,wins,losses,Seed
0,2024,1103,72.343750,66.812500,45.368830,7.500000,72.442244,12.312500,23.875000,8.187500,10.812500,10.312500,22.0,10.0,14
1,2024,1104,90.750000,81.062500,47.731660,11.062500,78.356164,15.906250,25.093750,11.031250,11.812500,11.281250,21.0,11.0,4
2,2024,1112,87.939394,72.242424,48.896195,7.818182,71.867008,18.575758,27.454545,11.545455,11.757576,12.848485,25.0,8.0,2
3,2024,1120,83.323529,68.029412,47.533849,8.029412,75.189873,17.911765,24.676471,9.794118,10.147059,12.000000,27.0,7.0,4
4,2024,1124,80.062500,71.250000,48.017621,8.562500,74.013605,14.687500,22.406250,9.406250,11.375000,10.906250,22.0,10.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2024,1443,79.100000,74.266667,46.587856,6.600000,71.941272,12.866667,27.100000,8.466667,13.333333,13.233333,19.0,11.0,15
64,2024,1447,61.689655,63.034483,38.549849,6.896552,70.654628,12.068966,22.586207,9.379310,8.931034,9.931034,14.0,15.0,16
65,2024,1450,74.303030,66.878788,46.649485,6.393939,70.081301,12.545455,25.424242,9.878788,10.151515,10.090909,24.0,9.0,7
66,2024,1458,75.057143,69.942857,46.279762,7.085714,75.887574,12.714286,22.085714,8.714286,9.228571,10.885714,22.0,13.0,5


In [46]:
teams

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2024
1,1102,Air Force,1985,2024
2,1103,Akron,1985,2024
3,1104,Alabama,1985,2024
4,1105,Alabama A&M,2000,2024
...,...,...,...,...
373,1474,Queens NC,2023,2024
374,1475,Southern Indiana,2023,2024
375,1476,Stonehill,2023,2024
376,1477,TX A&M Commerce,2023,2024


In [47]:
for i in tourney_teams['TeamID']:
    print(i, teams[teams['TeamID']==i]['TeamName'].iloc[0])

1103 Akron
1104 Alabama
1112 Arizona
1120 Auburn
1124 Baylor
1129 Boise St
1140 BYU
1155 Clemson
1158 Col Charleston
1159 Colgate
1160 Colorado
1161 Colorado St
1163 Connecticut
1166 Creighton
1173 Dayton
1179 Drake
1181 Duke
1182 Duquesne
1194 FL Atlantic
1196 Florida
1211 Gonzaga
1212 Grambling
1213 Grand Canyon
1222 Houston
1224 Howard
1228 Illinois
1235 Iowa St
1241 James Madison
1242 Kansas
1246 Kentucky
1253 Long Beach St
1255 Longwood
1266 Marquette
1270 McNeese St
1277 Michigan St
1280 Mississippi St
1286 Montana St
1287 Morehead St
1301 NC State
1304 Nebraska
1305 Nevada
1307 New Mexico
1314 North Carolina
1321 Northwestern
1324 Oakland
1332 Oregon
1345 Purdue
1355 S Dakota St
1359 Samford
1361 San Diego St
1376 South Carolina
1388 St Mary's CA
1389 St Peter's
1391 Stetson
1395 TCU
1397 Tennessee
1400 Texas
1401 Texas A&M
1403 Texas Tech
1412 UAB
1429 Utah St
1436 Vermont
1438 Virginia
1443 WKU
1447 Wagner
1450 Washington St
1458 Wisconsin
1463 Yale


In [48]:
teams[teams['TeamName']=='Akron']['TeamID'].iloc[0]

1103

In [49]:
predict_outcome(1304, 1401)

0

In [50]:
tourney_teams

,Season,TeamID,Pts,OPts,FG,FGM3,FT,Ast,DR,OR,TO,OTO,wins,losses,Seed
0,2024,1103,72.343750,66.812500,45.368830,7.500000,72.442244,12.312500,23.875000,8.187500,10.812500,10.312500,22.0,10.0,14
1,2024,1104,90.750000,81.062500,47.731660,11.062500,78.356164,15.906250,25.093750,11.031250,11.812500,11.281250,21.0,11.0,4
2,2024,1112,87.939394,72.242424,48.896195,7.818182,71.867008,18.575758,27.454545,11.545455,11.757576,12.848485,25.0,8.0,2
3,2024,1120,83.323529,68.029412,47.533849,8.029412,75.189873,17.911765,24.676471,9.794118,10.147059,12.000000,27.0,7.0,4
4,2024,1124,80.062500,71.250000,48.017621,8.562500,74.013605,14.687500,22.406250,9.406250,11.375000,10.906250,22.0,10.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2024,1443,79.100000,74.266667,46.587856,6.600000,71.941272,12.866667,27.100000,8.466667,13.333333,13.233333,19.0,11.0,15
64,2024,1447,61.689655,63.034483,38.549849,6.896552,70.654628,12.068966,22.586207,9.379310,8.931034,9.931034,14.0,15.0,16
65,2024,1450,74.303030,66.878788,46.649485,6.393939,70.081301,12.545455,25.424242,9.878788,10.151515,10.090909,24.0,9.0,7
66,2024,1458,75.057143,69.942857,46.279762,7.085714,75.887574,12.714286,22.085714,8.714286,9.228571,10.885714,22.0,13.0,5


In [51]:
predict_outcome(1124, 1112)

0

In [52]:
def predict_prob(team_1, team_2):
    first = tourney_teams[tourney_teams['TeamName']==team_1]
    second = tourney_teams[tourney_teams['TeamName']==team_2]
    game = first.merge(second, on='Season').drop(columns=['Season', 'TeamID_x', 'TeamID_y'])
    cols = np.array([])
    for c in game.columns:
        if c[-1] == 'x':
            cols = np.append(cols, c[:-1] + '1')
        else:
            cols = np.append(cols, c[:-1] + '2')
    game.columns = cols
    columns_1 = np.append(game.columns[12], game.columns[25])
    columns_2 = np.append(columns_1, game.columns[:12])
    columns_3 = np.append(columns_2, game.columns[13:25])
    game = game[list(columns_3)]
    return clf.predict_proba(game)[0][0], clf.predict_proba(game)[0][1]

In [53]:
def predict_results(team_name):
    for i in tourney_teams['TeamID']:
        team_id = teams[teams['TeamName']==team_name]['TeamID'].iloc[0]
        output = predict_outcome(team_id, i)
        print(teams[teams['TeamID']==i]['TeamName'].iloc[0], output)

In [54]:
predict_results("Creighton")

Akron 1
Alabama 0
Arizona 0
Auburn 0
Baylor 0
Boise St 1
BYU 0
Clemson 1
Col Charleston 1
Colgate 1
Colorado 1
Colorado St 1
Connecticut 0
Creighton 0
Dayton 1
Drake 1
Duke 0
Duquesne 1
FL Atlantic 0
Florida 0
Gonzaga 0
Grambling 1
Grand Canyon 1
Houston 0
Howard 1
Illinois 0
Iowa St 0
James Madison 1
Kansas 1
Kentucky 0
Long Beach St 1
Longwood 1
Marquette 0
McNeese St 0
Michigan St 0
Mississippi St 0
Montana St 1
Morehead St 1
NC State 0
Nebraska 1
Nevada 1
New Mexico 0
North Carolina 0
Northwestern 1
Oakland 1
Oregon 1
Purdue 0
S Dakota St 1
Samford 1
San Diego St 0
South Carolina 1
St Mary's CA 0
St Peter's 1
Stetson 1
TCU 0
Tennessee 0
Texas 0
Texas A&M 0
Texas Tech 0
UAB 1
Utah St 1
Vermont 1
Virginia 1
WKU 1
Wagner 1
Washington St 0
Wisconsin 0
Yale 1


In [55]:
get_team(1104)

,TeamID,TeamName,FirstD1Season,LastD1Season
3,1104,Alabama,1985,2024


In [56]:
tourney_teams.sort_values(by='FGM3')

,Season,TeamID,Pts,OPts,FG,FGM3,FT,Ast,DR,OR,TO,OTO,wins,losses,Seed
21,2024,1212,65.129032,68.870968,43.130006,5.322581,70.606532,9.161290,20.612903,7.741935,12.741935,12.032258,17.0,14.0,16
30,2024,1253,76.545455,75.939394,44.510386,5.424242,71.038961,15.030303,23.969697,10.454545,12.000000,12.575758,19.0,14.0,15
28,2024,1242,75.000000,69.096774,48.454188,5.677419,72.898799,18.483871,25.387097,7.161290,11.225806,11.161290,21.0,10.0,4
31,2024,1255,73.774194,69.677419,44.548458,5.677419,68.758717,12.516129,23.129032,11.838710,11.967742,12.870968,18.0,13.0,16
52,2024,1389,65.096774,63.225806,39.755530,5.838710,71.908397,10.806452,21.741935,10.483871,11.612903,12.419355,18.0,13.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,2024,1246,89.437500,79.687500,49.686142,9.937500,77.265501,17.562500,25.937500,8.281250,10.343750,11.593750,23.0,9.0,3
8,2024,1158,80.000000,72.939394,44.235294,10.545455,72.154116,15.272727,24.454545,11.181818,9.606061,10.787879,26.0,7.0,13
13,2024,1166,80.531250,69.718750,48.647242,10.593750,78.131635,17.125000,28.281250,7.125000,9.812500,7.468750,23.0,9.0,3
1,2024,1104,90.750000,81.062500,47.731660,11.062500,78.356164,15.906250,25.093750,11.031250,11.812500,11.281250,21.0,11.0,4
